## Limpeza de dados Cartola ano 2018

In [1]:
from pyspark.sql import HiveContext
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lower, col, lit, regexp_replace
import pandas as pd
import json
import requests

## Dados Agregados (Ainda não consegui interpretar)

In [ ]:
#Carrega arquivo CSV
dados_agregados_2018_df = spark.read.csv("/cartola/data/2018/2018_agregados.xlsx", header=True)
dados_agregados_2018_df.show()

## Jogadores (Erros que não consegui compreender)

In [6]:
#Carrega arquivo CSV
jogadores_2018_df = spark.read.csv("/cartola/data/2018/2018_jogadores.csv", header=True)
jogadores_2018_df.show()


+-----------------+-----------------+--------------------------+------------------+--------------------+
|atletas.atleta_id|  atletas.apelido|atletas.clube.id.full.name|atletas.posicao_id|        atletas.foto|
+-----------------+-----------------+--------------------------+------------------+--------------------+
|            37717|Fernando Henrique|                     Ceará|           Goleiro|https://s.glbimg....|
|            37657|      Edu Dracena|                 Palmeiras|          Zagueiro|https://s.glbimg....|
|            36540|             Juan|                  Flamengo|          Zagueiro|https://s.glbimg....|
|            37281|     Mano Menezes|                  Cruzeiro|           Técnico|https://s.glbimg....|
|            37306|    Gilson Kleina|               Chapecoense|           Técnico|https://s.glbimg....|
|            37674|           Wagner|                     Vasco|              Meia|https://s.glbimg....|
|            37656|            Fábio|                  

In [8]:
#Adiciona coluna ANO = 2018
jogadores_2018_ano = jogadores_2018_df.withColumn('ano', lit(2018))
jogadores_2018_ano.show()

+-----------------+-----------------+--------------------------+------------------+--------------------+----+
|atletas.atleta_id|  atletas.apelido|atletas.clube.id.full.name|atletas.posicao_id|        atletas.foto| ano|
+-----------------+-----------------+--------------------------+------------------+--------------------+----+
|            37717|Fernando Henrique|                     Ceará|           Goleiro|https://s.glbimg....|2018|
|            37657|      Edu Dracena|                 Palmeiras|          Zagueiro|https://s.glbimg....|2018|
|            36540|             Juan|                  Flamengo|          Zagueiro|https://s.glbimg....|2018|
|            37281|     Mano Menezes|                  Cruzeiro|           Técnico|https://s.glbimg....|2018|
|            37306|    Gilson Kleina|               Chapecoense|           Técnico|https://s.glbimg....|2018|
|            37674|           Wagner|                     Vasco|              Meia|https://s.glbimg....|2018|
|         

In [9]:
jogadores_2018_ano.write.partitionBy('ano').parquet('/cartola/2018/clean/jogadores')

In [10]:
#transforma arquivo em parquet
jogadores_2018_parquet = spark.read.parquet('/cartola/2018/clean/jogadores')

In [11]:
jogadores_2018_parquet.show()
jogadores_2018_parquet.schema


+-----------------+-----------------+--------------------------+------------------+--------------------+----+
|atletas.atleta_id|  atletas.apelido|atletas.clube.id.full.name|atletas.posicao_id|        atletas.foto| ano|
+-----------------+-----------------+--------------------------+------------------+--------------------+----+
|            37717|Fernando Henrique|                     Ceará|           Goleiro|https://s.glbimg....|2018|
|            37657|      Edu Dracena|                 Palmeiras|          Zagueiro|https://s.glbimg....|2018|
|            36540|             Juan|                  Flamengo|          Zagueiro|https://s.glbimg....|2018|
|            37281|     Mano Menezes|                  Cruzeiro|           Técnico|https://s.glbimg....|2018|
|            37306|    Gilson Kleina|               Chapecoense|           Técnico|https://s.glbimg....|2018|
|            37674|           Wagner|                     Vasco|              Meia|https://s.glbimg....|2018|
|         

StructType(List(StructField(atletas.atleta_id,StringType,true),StructField(atletas.apelido,StringType,true),StructField(atletas.clube.id.full.name,StringType,true),StructField(atletas.posicao_id,StringType,true),StructField(atletas.foto,StringType,true),StructField(ano,IntegerType,true)))

In [14]:
#Alterar ID para int.(está string)
jogadores_2018_parquet = jogadores_2018_parquet.withColumn("atletas.atleta_id", jogadores_2017_parquet["atletas.atleta_id"].cast(IntegerType()))

NameError: name 'jogadores_2017_parquet' is not defined

In [ ]:
jogadores_2017_parquet.schema

In [ ]:
jogadores_2017_parquet.show()

print(jogadores_2017_parquet.count())

## Medias-jogadores (Atender a esse arquivo)

In [17]:
medias_jogadores_2018_df = spark.read.csv("/cartola/data/2018/2018-medias-jogadores.csv", header=True)
medias_jogadores_2018_df_ano = medias_jogadores_2018_df.withColumn('ano', lit(2018))

In [19]:
medias_jogadores_2018_df_ano.show()

<bound method DataFrame.show of DataFrame[player_slug: string, player_id: string, player_nickname: string, player_team: string, player_position: string, price_cartoletas: string, score_mean: string, score_no_cleansheets_mean: string, diff_home_away_s: string, n_games: string, score_mean_home: string, score_mean_away: string, shots_x_mean: string, fouls_mean: string, RB_mean: string, PE_mean: string, A_mean: string, I_mean: string, FS_mean: string, FF_mean: string, G_mean: string, DD_mean: string, DP_mean: string, ano: int]>

## Partidas

In [20]:
partidas_2018_df = spark.read.csv("/cartola/data/2018/2018_partidas.csv", header=True)

In [21]:
partidas_2018_df_ano = partidas_2018_df.withColumn('ano', lit(2018))

In [22]:
partidas_2018_df_ano.show(1000)

+----+-----+------------------+------------------+------+------------------+--------------------+----+
|game|round|              date|         home_team| score|         away_team|               arena| ano|
+----+-----+------------------+------------------+------+------------------+--------------------+----+
|   1|    1|14/04/2018 - 16:00|     Cruzeiro - MG| 0 x 1|       Grêmio - RS|Mineirão - Belo H...|2018|
|   2|    1|15/04/2018 - 19:00|     Atlético - PR| 5 x 1|  Chapecoense - SC|Arena da Baixada ...|2018|
|   3|    1|15/04/2018 - 11:00|      América - MG| 3 x 0|        Sport - PE|Independência - B...|2018|
|   4|    1|14/04/2018 - 19:00|      Vitória - BA| 2 x 2|     Flamengo - RJ|Manoel Barradas -...|2018|
|   5|    1|15/04/2018 - 16:00|Vasco da Gama - RJ| 2 x 1|     Atlético - MG|São Januário - Ri...|2018|
|   6|    1|16/04/2018 - 20:00|     Botafogo - RJ| 1 x 1|    Palmeiras - SP|Nilton Santos - R...|2018|
|   7|    1|16/04/2018 - 20:00|    São Paulo - SP| 1 x 0|       Paraná - 

In [23]:
partidas_2018_df_ano.write.partitionBy('ano').parquet('/cartola/2018/clean/2018_partidas')

In [24]:
partidas_2018_parquet = spark.read.parquet('/cartola/2018/clean/2018_partidas')

In [25]:
partidas_2018_parquet.show(10000)
partidas_2018_parquet.schema

+----+-----+------------------+------------------+------+------------------+--------------------+----+
|game|round|              date|         home_team| score|         away_team|               arena| ano|
+----+-----+------------------+------------------+------+------------------+--------------------+----+
|   1|    1|14/04/2018 - 16:00|     Cruzeiro - MG| 0 x 1|       Grêmio - RS|Mineirão - Belo H...|2018|
|   2|    1|15/04/2018 - 19:00|     Atlético - PR| 5 x 1|  Chapecoense - SC|Arena da Baixada ...|2018|
|   3|    1|15/04/2018 - 11:00|      América - MG| 3 x 0|        Sport - PE|Independência - B...|2018|
|   4|    1|14/04/2018 - 19:00|      Vitória - BA| 2 x 2|     Flamengo - RJ|Manoel Barradas -...|2018|
|   5|    1|15/04/2018 - 16:00|Vasco da Gama - RJ| 2 x 1|     Atlético - MG|São Januário - Ri...|2018|
|   6|    1|16/04/2018 - 20:00|     Botafogo - RJ| 1 x 1|    Palmeiras - SP|Nilton Santos - R...|2018|
|   7|    1|16/04/2018 - 20:00|    São Paulo - SP| 1 x 0|       Paraná - 

StructType(List(StructField(game,StringType,true),StructField(round,StringType,true),StructField(date,StringType,true),StructField(home_team,StringType,true),StructField(score,StringType,true),StructField(away_team,StringType,true),StructField(arena,StringType,true),StructField(ano,IntegerType,true)))

In [27]:
partidas_2018_parquet = partidas_2018_parquet.withColumn("game", partidas_2018_parquet["game"].cast(IntegerType()))
partidas_2018_parquet = partidas_2018_parquet.withColumn("round", partidas_2018_parquet["round"].cast(IntegerType()))

In [28]:
partidas_2018_parquet.show()

print(partidas_2018_parquet.count())

+----+-----+------------------+------------------+-----+------------------+--------------------+----+
|game|round|              date|         home_team|score|         away_team|               arena| ano|
+----+-----+------------------+------------------+-----+------------------+--------------------+----+
|   1|    1|14/04/2018 - 16:00|     Cruzeiro - MG|0 x 1|       Grêmio - RS|Mineirão - Belo H...|2018|
|   2|    1|15/04/2018 - 19:00|     Atlético - PR|5 x 1|  Chapecoense - SC|Arena da Baixada ...|2018|
|   3|    1|15/04/2018 - 11:00|      América - MG|3 x 0|        Sport - PE|Independência - B...|2018|
|   4|    1|14/04/2018 - 19:00|      Vitória - BA|2 x 2|     Flamengo - RJ|Manoel Barradas -...|2018|
|   5|    1|15/04/2018 - 16:00|Vasco da Gama - RJ|2 x 1|     Atlético - MG|São Januário - Ri...|2018|
|   6|    1|16/04/2018 - 20:00|     Botafogo - RJ|1 x 1|    Palmeiras - SP|Nilton Santos - R...|2018|
|   7|    1|16/04/2018 - 20:00|    São Paulo - SP|1 x 0|       Paraná - PR|Morumbi

## 2018_tabelas

In [33]:
tabelas_2018_df = spark.read.csv("/cartola/data/2018/2018_tabelas.csv", header=True)

In [34]:
tabelas_2018_df_ano = tabelas_2018_df.withColumn('ano', lit(2018))

In [35]:
tabelas_2018_df_ano.show()

+---+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|Pos|             Clube|  P|  J|  V|  E|  D| GP| GC| SG| VM| VV| DM| DV| CA| CV|  %| ano|
+---+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
| 1º|     Flamengo - RJ| 10|  5|  3|  1|  1| 11|  5|  6|  2|  1|  0|  1| 13|  1| 66|2018|
| 2º|  Corinthians - SP| 10|  5|  3|  1|  1|  8|  3|  5|  2|  1|  0|  1|  8|  0| 66|2018|
| 3º|     Atlético - MG| 10|  5|  3|  1|  1|  8|  6|  2|  2|  1|  0|  1| 15|  0| 66|2018|
| 4º|       Grêmio - RS|  8|  5|  2|  2|  1|  7|  3|  4|  1|  1|  0|  1|  9|  1| 53|2018|
| 5º|    Palmeiras - SP|  8|  5|  2|  2|  1|  5|  3|  2|  1|  1|  0|  1| 15|  0| 53|2018|
| 6º|     Botafogo - RJ|  8|  5|  2|  2|  1|  6|  5|  1|  2|  0|  0|  1| 11|  0| 53|2018|
| 7º|Vasco da Gama - RJ|  7|  4|  2|  1|  1|  9|  6|  3|  2|  0|  1|  0| 12|  0| 58|2018|
| 8º|     Cruzeiro - MG|  7|  5|  2|  1|  2|  3|  2|  1|  2|  0|  1|  1| 11|  0| 46|2018|
| 9º|   Fl

In [36]:
tabelas_2018_df_ano.write.partitionBy('ano').parquet('/cartola/2018/clean/tabelas')

In [37]:
tabelas_2018_parquet = spark.read.parquet("/cartola/2018/clean/tabelas")

In [38]:
tabelas_2018_parquet.show()
tabelas_2018_parquet.schema

+---+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|Pos|             Clube|  P|  J|  V|  E|  D| GP| GC| SG| VM| VV| DM| DV| CA| CV|  %| ano|
+---+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
| 1º|     Flamengo - RJ| 10|  5|  3|  1|  1| 11|  5|  6|  2|  1|  0|  1| 13|  1| 66|2018|
| 2º|  Corinthians - SP| 10|  5|  3|  1|  1|  8|  3|  5|  2|  1|  0|  1|  8|  0| 66|2018|
| 3º|     Atlético - MG| 10|  5|  3|  1|  1|  8|  6|  2|  2|  1|  0|  1| 15|  0| 66|2018|
| 4º|       Grêmio - RS|  8|  5|  2|  2|  1|  7|  3|  4|  1|  1|  0|  1|  9|  1| 53|2018|
| 5º|    Palmeiras - SP|  8|  5|  2|  2|  1|  5|  3|  2|  1|  1|  0|  1| 15|  0| 53|2018|
| 6º|     Botafogo - RJ|  8|  5|  2|  2|  1|  6|  5|  1|  2|  0|  0|  1| 11|  0| 53|2018|
| 7º|Vasco da Gama - RJ|  7|  4|  2|  1|  1|  9|  6|  3|  2|  0|  1|  0| 12|  0| 58|2018|
| 8º|     Cruzeiro - MG|  7|  5|  2|  1|  2|  3|  2|  1|  2|  0|  1|  1| 11|  0| 46|2018|
| 9º|   Fl

StructType(List(StructField(Pos,StringType,true),StructField(Clube,StringType,true),StructField(P,StringType,true),StructField(J,StringType,true),StructField(V,StringType,true),StructField(E,StringType,true),StructField(D,StringType,true),StructField(GP,StringType,true),StructField(GC,StringType,true),StructField(SG,StringType,true),StructField(VM,StringType,true),StructField(VV,StringType,true),StructField(DM,StringType,true),StructField(DV,StringType,true),StructField(CA,StringType,true),StructField(CV,StringType,true),StructField(%,StringType,true),StructField(ano,IntegerType,true)))

In [40]:
#deve-se converter valores em BigDecimal?
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("Pos", tabelas_2018_parquet["Pos"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("P", tabelas_2018_parquet["P"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("J", tabelas_2018_parquet["J"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("E", tabelas_2018_parquet["E"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("D", tabelas_2018_parquet["D"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("GP", tabelas_2018_parquet["GP"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("GC", tabelas_2018_parquet["GC"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("SG", tabelas_2018_parquet["SG"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("VM", tabelas_2018_parquet["VM"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("VV", tabelas_2018_parquet["VV"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("DM", tabelas_2018_parquet["DM"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("DV", tabelas_2018_parquet["DV"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("CA", tabelas_2018_parquet["CA"].cast(IntegerType()))
tabelas_2018_parquet = tabelas_2018_parquet.withColumn("CV", tabelas_2018_parquet["CV"].cast(IntegerType()))

In [41]:
tabelas_2018_parquet.schema

+----+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
| Pos|             Clube|  P|  J|  V|  E|  D| GP| GC| SG| VM| VV| DM| DV| CA| CV|  %| ano|
+----+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|null|     Flamengo - RJ| 10|  5|  3|  1|  1| 11|  5|  6|  2|  1|  0|  1| 13|  1| 66|2018|
|null|  Corinthians - SP| 10|  5|  3|  1|  1|  8|  3|  5|  2|  1|  0|  1|  8|  0| 66|2018|
|null|     Atlético - MG| 10|  5|  3|  1|  1|  8|  6|  2|  2|  1|  0|  1| 15|  0| 66|2018|
|null|       Grêmio - RS|  8|  5|  2|  2|  1|  7|  3|  4|  1|  1|  0|  1|  9|  1| 53|2018|
|null|    Palmeiras - SP|  8|  5|  2|  2|  1|  5|  3|  2|  1|  1|  0|  1| 15|  0| 53|2018|
|null|     Botafogo - RJ|  8|  5|  2|  2|  1|  6|  5|  1|  2|  0|  0|  1| 11|  0| 53|2018|
|null|Vasco da Gama - RJ|  7|  4|  2|  1|  1|  9|  6|  3|  2|  0|  1|  0| 12|  0| 58|2018|
|null|     Cruzeiro - MG|  7|  5|  2|  1|  2|  3|  2|  1|  2|  0|  1|  1| 11|  0| 46|2018|

StructType(List(StructField(Pos,IntegerType,true),StructField(Clube,StringType,true),StructField(P,IntegerType,true),StructField(J,IntegerType,true),StructField(V,StringType,true),StructField(E,IntegerType,true),StructField(D,IntegerType,true),StructField(GP,IntegerType,true),StructField(GC,IntegerType,true),StructField(SG,IntegerType,true),StructField(VM,IntegerType,true),StructField(VV,IntegerType,true),StructField(DM,IntegerType,true),StructField(DV,IntegerType,true),StructField(CA,IntegerType,true),StructField(CV,IntegerType,true),StructField(%,StringType,true),StructField(ano,IntegerType,true)))

In [43]:
tabelas_2018_parquet.show()
print(tabelas_2018_parquet.count())

+----+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
| Pos|             Clube|  P|  J|  V|  E|  D| GP| GC| SG| VM| VV| DM| DV| CA| CV|  %| ano|
+----+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|null|     Flamengo - RJ| 10|  5|  3|  1|  1| 11|  5|  6|  2|  1|  0|  1| 13|  1| 66|2018|
|null|  Corinthians - SP| 10|  5|  3|  1|  1|  8|  3|  5|  2|  1|  0|  1|  8|  0| 66|2018|
|null|     Atlético - MG| 10|  5|  3|  1|  1|  8|  6|  2|  2|  1|  0|  1| 15|  0| 66|2018|
|null|       Grêmio - RS|  8|  5|  2|  2|  1|  7|  3|  4|  1|  1|  0|  1|  9|  1| 53|2018|
|null|    Palmeiras - SP|  8|  5|  2|  2|  1|  5|  3|  2|  1|  1|  0|  1| 15|  0| 53|2018|
|null|     Botafogo - RJ|  8|  5|  2|  2|  1|  6|  5|  1|  2|  0|  0|  1| 11|  0| 53|2018|
|null|Vasco da Gama - RJ|  7|  4|  2|  1|  1|  9|  6|  3|  2|  0|  1|  0| 12|  0| 58|2018|
|null|     Cruzeiro - MG|  7|  5|  2|  1|  2|  3|  2|  1|  2|  0|  1|  1| 11|  0| 46|2018|

## DEMONSTRAÇÃO

In [ ]:
partidas_2014_df = spark.read.csv("/cartola/data/2014/2014_jogadores.csv", header=True)
partidas_2014_ct = partidas_2014_df.withColumn('time', regexp_replace('home_team', ' - RJ', ''))
final_partidas = partidas_2014_ct.withColumn('time_low', lower(col('time'))).show(truncate=False)

In [ ]:
with_ano_partidas = partidas_2014_ct.withColumn('ano', lit(2014)).show(truncate=False)
with_ano_partidas.show()